# Get Data

In [9]:
ls

Combined_News_DJIA.csv  RedditNews.csv          upload_DJIA_table.csv
README.md               financialnews.ipynb


In [208]:
pd.read_csv('RedditNews.csv')

,Date,News
0,2016-07-01,A 117-year-old woman in Mexico City finally re...
1,2016-07-01,IMF chief backs Athens as permanent Olympic host
2,2016-07-01,"The president of France says if Brexit won, so..."
3,2016-07-01,British Man Who Must Give Police 24 Hours' Not...
4,2016-07-01,100+ Nobel laureates urge Greenpeace to stop o...
...,...,...
73603,2008-06-08,b'Man goes berzerk in Akihabara and stabs ever...
73604,2008-06-08,b'Threat of world AIDS pandemic among heterose...
73605,2008-06-08,b'Angst in Ankara: Turkey Steers into a Danger...
73606,2008-06-08,"b""UK: Identity cards 'could be used to spy on ..."


In [209]:
pd.read_csv('upload_DJIA_table.csv')

,Date,Open,High,Low,Close,Volume,Adj Close
0,2016-07-01,17924.240234,18002.380859,17916.910156,17949.369141,82160000,17949.369141
1,2016-06-30,17712.759766,17930.609375,17711.800781,17929.990234,133030000,17929.990234
2,2016-06-29,17456.019531,17704.509766,17456.019531,17694.679688,106380000,17694.679688
3,2016-06-28,17190.509766,17409.720703,17190.509766,17409.720703,112190000,17409.720703
4,2016-06-27,17355.210938,17355.210938,17063.080078,17140.240234,138740000,17140.240234
...,...,...,...,...,...,...,...
1984,2008-08-14,11532.070312,11718.280273,11450.889648,11615.929688,159790000,11615.929688
1985,2008-08-13,11632.809570,11633.780273,11453.339844,11532.959961,182550000,11532.959961
1986,2008-08-12,11781.700195,11782.349609,11601.519531,11642.469727,173590000,11642.469727
1987,2008-08-11,11729.669922,11867.110352,11675.530273,11782.349609,183190000,11782.349609


In [5]:
import pandas as pd

In [131]:
import numpy as np

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression


In [46]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

In [11]:
df = pd.read_csv('Combined_News_DJIA.csv')

In [186]:
#Train Data- all dates before 2015
train = df[df['Date']< '2015-01-01']

In [187]:
#test data is the last year
test = df[df['Date'] >= '2015-01-01']

In [197]:
train['Label'].value_counts()

1    873
0    738
Name: Label, dtype: int64

In [198]:
test['Label'].value_counts()

1    192
0    186
Name: Label, dtype: int64

#### To do explain loc and iloc

In [19]:
example = train.iloc[3,10]

In [21]:
example = example.lower()

In [23]:
example = CountVectorizer().build_tokenizer()(example)

# Log Reg on Data

### Vectorize

In [188]:
trainheadlines = []
for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))


In [189]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

(1611, 31675)


In [190]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

/Users/johnrick/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [191]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)
predictions = basicmodel.predict(basictest)

In [192]:
pd.crosstab(test["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,61,125
1,92,100


In [199]:
basicwords = basicvectorizer.get_feature_names()
basiccoeffs = basicmodel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
coeffdf.head(10)


,Word,Coefficient
19419,nigeria,0.497924
25261,self,0.452526
29286,tv,0.428011
15998,korea,0.425863
20135,olympics,0.425716
15843,kills,0.411636
26323,so,0.411267
29256,turn,0.394855
10874,fears,0.388555
28274,territory,0.384031


In [200]:
coeffdf.tail(10)

,Word,Coefficient
27299,students,-0.424441
8478,did,-0.427079
6683,congo,-0.431925
12818,hacking,-0.444069
7139,country,-0.448570
16949,low,-0.463116
3651,begin,-0.470454
25433,sex,-0.494555
24754,sanctions,-0.549725
24542,run,-0.587794


In [201]:
advancedvectorizer = CountVectorizer(ngram_range=(2,2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)


In [202]:
print(advancedtrain.shape)

(1611, 366721)


In [203]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

/Users/johnrick/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [204]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
advpredictions = advancedmodel.predict(advancedtest)


In [205]:
pd.crosstab(test["Label"], advpredictions, rownames=["Actual"], colnames=["Predicted"])


Predicted,0,1
Actual,,
0,66,120
1,45,147


In [206]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(10)


,Words,Coefficient
272047,right to,0.286533
24710,and other,0.275274
285392,set to,0.274698
316194,the first,0.262873
157511,in china,0.227943
159522,in south,0.224184
125870,found in,0.219130
124411,forced to,0.216726
173246,it has,0.211137
322590,this is,0.209239


In [207]:
advcoeffdf.tail(10)


,Words,Coefficient
326846,to help,-0.198495
118707,fire on,-0.201654
155038,if he,-0.209702
242528,people are,-0.211303
31669,around the,-0.213362
321333,there is,-0.215699
327113,to kill,-0.221812
340714,up in,-0.226289
358917,with iran,-0.227516
315485,the country,-0.331153


In [100]:
logreg = LogisticRegression()

In [106]:
logreg.fit(X_train, y_train)

/Users/johnrick/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [79]:
test_headlines= []

In [80]:
for row in range(0, len(test.index)):
    test_headlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))

In [81]:
len(test_headlines)

378

In [87]:
X_test = count_vectorize.transform(test_headlines)

In [88]:
X_test

<378x16127 sparse matrix of type '<class 'numpy.int64'>'
	with 119804 stored elements in Compressed Sparse Row format>

In [108]:
#the 
y_test = test['Label'].values

In [91]:
X_train

<1611x31675 sparse matrix of type '<class 'numpy.int64'>'
	with 489193 stored elements in Compressed Sparse Row format>

In [92]:
X_test

<378x16127 sparse matrix of type '<class 'numpy.int64'>'
	with 119804 stored elements in Compressed Sparse Row format>

In [107]:
y_train

array([0, 1, 0, ..., 0, 0, 0])

In [109]:
y_test

array([1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,

In [110]:
y_pred_log_reg = logreg.predict(X_test)

ValueError: X has 16127 features per sample; expecting 31675

In [55]:
y_test

<378x31675 sparse matrix of type '<class 'numpy.int64'>'
	with 116088 stored elements in Compressed Sparse Row format>

#### Learn Crosstab

In [45]:
pd.crosstab(test['Label'], predictions, rownames= ['Actual'], colnames = ['Predicted'])

Predicted,0,1
Actual,,
0,61,125
1,92,100


In [54]:
y_test

<378x31675 sparse matrix of type '<class 'numpy.int64'>'
	with 116088 stored elements in Compressed Sparse Row format>

In [53]:
print('Logistic Regression:')
print(classification_report(y_test, y_pred_log_reg))

Logistic Regression:


TypeError: len() of unsized object

In [161]:
X_train = train_headlines

In [175]:
y_train = train['Label']

In [179]:
X_test = test_headlines

In [164]:
y_test = test['Label']

In [165]:
vectorizer = CountVectorizer()

In [166]:
#no numpy - just list
X_train = vectorizer.fit_transform(X_train)

In [168]:
X_train

<1611x31675 sparse matrix of type '<class 'numpy.int64'>'
	with 489193 stored elements in Compressed Sparse Row format>

In [180]:
X_test = vectorizer.fit_transform(X_test)

In [171]:
y_test

1611    1
1612    0
1613    0
1614    1
1615    1
       ..
1984    0
1985    1
1986    1
1987    1
1988    1
Name: Label, Length: 378, dtype: int64

In [172]:
logreg = LogisticRegression()

In [176]:
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [178]:
X_test

<378x16127 sparse matrix of type '<class 'numpy.int64'>'
	with 119804 stored elements in Compressed Sparse Row format>

In [181]:
y_pred_log_reg = logreg.predict(X_test)

ValueError: X has 16127 features per sample; expecting 31675

In [ ]:
'''
https://www.kaggle.com/ndrewgele/omg-nlp-with-the-djia-and-reddit

'''